In [25]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score


In [26]:
labelled_data = pd.read_csv('labelled_whitelist_shop.csv')
labelled_data.shape

(536, 22)

In [27]:
labelled_data.info()
labelled_data.tail(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   manage_shop_indicator          536 non-null    float64
 1   officialstore_indicator        536 non-null    float64
 2   preferred_shop_indicator       536 non-null    float64
 3   crossborder_indicator          536 non-null    float64
 4   shop_category                  536 non-null    float64
 5   new_seller_flag                536 non-null    float64
 6   seller_centre_login_L30D       536 non-null    float64
 7   shop_sku_number                536 non-null    float64
 8   shop_follower_number           536 non-null    float64
 9   shop_L180D_order               536 non-null    float64
 10  weighted_shop_rating           536 non-null    float64
 11  performance_date               536 non-null    float64
 12  masked_item_impression         536 non-null    flo

,manage_shop_indicator,officialstore_indicator,preferred_shop_indicator,crossborder_indicator,shop_category,new_seller_flag,seller_centre_login_L30D,shop_sku_number,shop_follower_number,shop_L180D_order,...,masked_item_impression,masked_order,masked_shop_page_view,masked_shop_click_from_search,masked_campaign_tab_click,masked_other_tab_click,kmeans.labels,agglomerative.average.labels,agglomerative.complete.labels,meanshift.labels
516,0.0,0.0,0.0,0.0,9.0,0.0,30.0,731.0,1080.0,405.0,...,7.463074e+05,53.357143,0.642857,7.082707,11.0,30.665414,0,3,0,0
517,0.0,0.0,0.0,0.0,4.0,1.0,6.0,324.0,972.0,405.0,...,7.991000e+03,2.571429,113.928571,3.169173,11.0,33.932331,0,3,0,0
518,0.0,0.0,0.0,0.0,6.0,0.0,26.0,2172.0,1095.0,405.0,...,6.264412e+06,552.285714,284.857143,21.759398,11.0,86.120301,0,3,0,0
519,0.0,0.0,0.0,1.0,9.0,0.0,30.0,11060.0,990.0,405.0,...,1.222253e+07,5.714286,0.071429,56.541353,11.0,58.868421,0,3,0,0
520,0.0,0.0,0.0,1.0,13.0,0.0,24.0,1490.0,1038.0,405.0,...,2.209722e+06,12.928571,0.000000,24.090226,11.0,47.041353,0,3,0,0
521,0.0,0.0,0.0,0.0,0.0,0.0,27.0,1083.0,1311.0,405.0,...,2.152701e+08,4804.785714,48.642857,406.522556,11.0,114.488722,0,3,0,0
522,0.0,0.0,0.0,0.0,13.0,0.0,30.0,368.0,1617.0,405.0,...,1.571983e+09,33961.071429,0.857143,3020.206767,11.0,111.428571,0,3,0,0
523,0.0,0.0,0.0,1.0,13.0,0.0,25.0,11247.0,990.0,405.0,...,3.896590e+06,4.357143,0.000000,18.657895,11.0,58.703008,0,3,0,0
524,0.0,0.0,0.0,1.0,9.0,0.0,30.0,5967.0,1023.0,405.0,...,9.073076e+05,0.642857,0.428571,10.507519,11.0,43.774436,0,3,0,0
525,0.0,0.0,0.0,1.0,0.0,0.0,27.0,445.0,981.0,405.0,...,1.431181e+05,2.500000,53.285714,6.872180,11.0,29.631579,0,3,0,0


In [71]:
#decision tree
    #kmeans label
x = labelled_data.iloc[:,:-4]
# x.tail(20)
y = labelled_data.iloc[:,-4]
# y.tail(20)
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.4)
dtc = DecisionTreeClassifier()
dtc = dtc.fit(train_x,train_y)
predict_y = dtc.predict(test_x)
print('decision_tree result->kmeans label')
print("mean_square_error",mean_squared_error(test_y,predict_y))
print("accuracy_score",accuracy_score(test_y,predict_y))
# dtc.predict_proba(test_x)
print("-------------------------")
    #agglomerative.average.labels
x1 = labelled_data.iloc[:,:-4]
# x.tail(20)
y1 = labelled_data.iloc[:,-3]
# y.tail(20)
train_x1,test_x1,train_y1,test_y1 = train_test_split(x1,y1,test_size = 0.4)
dtc1 = DecisionTreeClassifier()
dtc1 = dtc.fit(train_x1,train_y1)
predict_y1 = dtc.predict(test_x1)
print('decision_tree result->agglomerative.average.labels')
print("mean_square_error",mean_squared_error(test_y1,predict_y1))
print("accuracy_score",accuracy_score(test_y1,predict_y1))
# dtc1.predict_proba(test_x1)

print("-------------------------")
    #agglomerative.complete.labels
x2 = labelled_data.iloc[:,:-4]
# x.tail(20)
y2 = labelled_data.iloc[:,-2]
# y.tail(20)
train_x2,test_x2,train_y2,test_y2 = train_test_split(x2,y2,test_size = 0.4)
dtc2 = DecisionTreeClassifier()
dtc2 = dtc.fit(train_x2,train_y2)
predict_y2 = dtc.predict(test_x2)
print('decision_tree result->agglomerative.complete.labels')
print("mean_square_error",mean_squared_error(test_y2,predict_y2))
print("accuracy_score",accuracy_score(test_y2,predict_y2))
# dtc1.predict_proba(test_x1)

print("-------------------------")
    #meanshift.labels 
x3 = labelled_data.iloc[:,:-4]
# x.tail(20)
y3 = labelled_data.iloc[:,-1]
# y.tail(20)
train_x3,test_x3,train_y3,test_y3 = train_test_split(x3,y3,test_size = 0.4)
dtc3 = DecisionTreeClassifier()
dtc3 = dtc.fit(train_x3,train_y3)
predict_y3 = dtc.predict(test_x3)
print('decision_tree result->meanshift.labels ')
print("mean_square_error",mean_squared_error(test_y3,predict_y3))
print("accuracy_score",accuracy_score(test_y3,predict_y3))

decision_tree result->kmeans label
mean_square_error 0.05581395348837209
accuracy_score 0.9813953488372092
-------------------------
decision_tree result->agglomerative.average.labels
mean_square_error 0.046511627906976744
accuracy_score 0.9906976744186047
-------------------------
decision_tree result->agglomerative.complete.labels
mean_square_error 0.08372093023255814
accuracy_score 0.9906976744186047
-------------------------
decision_tree result->meanshift.labels 
mean_square_error 0.5488372093023256
accuracy_score 0.9767441860465116


In [39]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import SVC
# x = labelled_data.iloc[:,:-4]
# # x.tail(20)
# y = labelled_data.iloc[:,-4]
# # y.tail(20)
# train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.4)
# clf = OneVsRestClassifier(SVC()).fit(train_x,train_y)
# clf.predict(test_x)
# clf.predict_prob(test_x)

AttributeError: 'OneVsRestClassifier' object has no attribute 'predict_prob'

In [70]:
#logistics regression
from matplotlib import pyplot
import scipy as sp
import numpy as np
from matplotlib import pylab
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import time

def plot_pr(auc_score, precision, recall, label=None):
    pylab.figure(num=None, figsize=(6, 5))
    pylab.xlim([0.0, 1.0])
    pylab.ylim([0.0, 1.0])
    pylab.xlabel('Recall')
    pylab.ylabel('Precision')
    pylab.title('P/R (AUC=%0.2f) / %s' % (auc_score, label))
    pylab.fill_between(recall, precision, alpha=0.5)
    pylab.grid(True, linestyle='-', color='0.75')
    pylab.plot(recall, precision, lw=1)    
    pylab.show()

    
labelled_data = pd.read_csv('labelled_whitelist_shop.csv')
x = labelled_data.iloc[:,:-4]
y = labelled_data.iloc[:,-4]

In [68]:
non_whitelist = pd.read_csv('non_whitelist_filled.csv')
# non_whitelist.head(20)
non_whitelist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364252 entries, 0 to 364251
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   manage_shop_indicator          364252 non-null  int64  
 1   officialstore_indicator        364252 non-null  int64  
 2   preferred_shop_indicator       364252 non-null  int64  
 3   crossborder_indicator          364252 non-null  int64  
 4   shop_category                  364252 non-null  int64  
 5   new_seller_flag                364252 non-null  int64  
 6   seller_centre_login_L30D       364252 non-null  int64  
 7   shop_sku_number                364252 non-null  int64  
 8   shop_follower_number           364252 non-null  int64  
 9   shop_L180D_order               364252 non-null  int64  
 10  weighted_shop_rating           364252 non-null  float64
 11  performance_date               364252 non-null  int64  
 12  masked_item_impression        

In [74]:
#dtc
pred_result = dtc.predict(non_whitelist.tail(20))
print(pred_result)
pred_prob = dtc.predict_proba(non_whitelist.tail(20))
print(pred_prob)
print("-----------------------")
#dtc1
pred_result = dtc1.predict(non_whitelist.tail(20))
print(pred_result)
pred_prob = dtc1.predict_proba(non_whitelist.tail(20))
print(pred_prob)
#dtc2
pred_result = dtc2.predict(non_whitelist.tail(20))
print(pred_result)
pred_prob = dtc2.predict_proba(non_whitelist.tail(20))
print(pred_prob)
print("-----------------------")
#dtc3
pred_result = dtc3.predict(non_whitelist.tail(20))
print(pred_result)
pred_prob = dtc3.predict_proba(non_whitelist.tail(20))
print(pred_prob)
print("-----------------------")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
-----------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0